<a href="https://colab.research.google.com/github/alortiz05/DDDS-Cohort-16-Projects/blob/main/3M_IMBD_Data_Pull_Final_Part_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
import requests #to import http requests

#Step 2: List of IMDb dataset URLs (hosted by IMDb as .tsv.gz compressed files)
imdb_files = [
    'https://datasets.imdbws.com/name.basics.tsv.gz',
    'https://datasets.imdbws.com/title.akas.tsv.gz',
    'https://datasets.imdbws.com/title.basics.tsv.gz',
    'https://datasets.imdbws.com/title.crew.tsv.gz',
    'https://datasets.imdbws.com/title.episode.tsv.gz',
    'https://datasets.imdbws.com/title.principals.tsv.gz',
    'https://datasets.imdbws.com/title.ratings.tsv.gz'
]
# Step 3: Print the size of each file by sending a HEAD request to get metadata (without downloading full content)
print("IMDb file sizes (compressed):")
for url in imdb_files:
    try:
        response = requests.head(url, allow_redirects=True) # Follow redirects if any
        size_bytes = int(response.headers.get('Content-Length', 0))# Get size in bytes
        size_mb = size_bytes / (1024 * 1024)# Convert to megabytes
        print(f"{url.split('/')[-1]}: {size_mb:.2f} MB") # Display the filename and size
    except Exception as e:
        print(f"Failed to get size for {url}: {e}")# Handle network or header issues

IMDb file sizes (compressed):
name.basics.tsv.gz: 272.82 MB
title.akas.tsv.gz: 434.32 MB
title.basics.tsv.gz: 197.64 MB
title.crew.tsv.gz: 73.09 MB
title.episode.tsv.gz: 47.49 MB
title.principals.tsv.gz: 686.94 MB
title.ratings.tsv.gz: 7.58 MB


In [ ]:
# Import DuckDB for SQL queries and os for file handling
import duckdb
import os

# Import Google Colab drive module and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to save IMDb datasets inside your Drive
download_dir = "//content/drive/MyDrive/Capstone Project/imdb_data"  # Save directly to your Drive

# Create the directory if it doesn't already exist
os.makedirs(download_dir, exist_ok=True)

# IMDb's public dataset URLs (as .tsv.gz compressed files)
imdb_files = [
    'https://datasets.imdbws.com/name.basics.tsv.gz',
    'https://datasets.imdbws.com/title.akas.tsv.gz',
    'https://datasets.imdbws.com/title.basics.tsv.gz',
    'https://datasets.imdbws.com/title.crew.tsv.gz',
    'https://datasets.imdbws.com/title.episode.tsv.gz',
    'https://datasets.imdbws.com/title.principals.tsv.gz',
    'https://datasets.imdbws.com/title.ratings.tsv.gz'
]

# Download each IMDb file if it doesn't already exist in the target folder
for url in imdb_files:
    file_name = os.path.join(download_dir, url.split('/')[-1]) # e.g., title.basics.tsv.gz
    if os.path.exists(file_name):
        print(f"{file_name} already exists.")
        continue
    print(f"Downloading {url} ...")
    response = requests.get(url, stream=True)

     # Save the file in chunks to avoid memory issues
    with open(file_name, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"Saved to {file_name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
//content/drive/MyDrive/Capstone Project/imdb_data/name.basics.tsv.gz already exists.
//content/drive/MyDrive/Capstone Project/imdb_data/title.akas.tsv.gz already exists.
//content/drive/MyDrive/Capstone Project/imdb_data/title.basics.tsv.gz already exists.
//content/drive/MyDrive/Capstone Project/imdb_data/title.crew.tsv.gz already exists.
//content/drive/MyDrive/Capstone Project/imdb_data/title.episode.tsv.gz already exists.
//content/drive/MyDrive/Capstone Project/imdb_data/title.principals.tsv.gz already exists.
//content/drive/MyDrive/Capstone Project/imdb_data/title.ratings.tsv.gz already exists.


- did not use title_episode because we are looking at movies
- made sure to not pull in multiple tconst
-

In [ ]:

# Connect to DuckDB in-memory (or specify a file for persistent DB)
# Connect to DuckDB in memory (data will not persist after session ends)
con = duckdb.connect()

# Ensure download_dir points to the location where files were saved
# This should match the download_dir used in the previous cell
# Define path to IMDb data files stored in Google Drive
download_dir = "/content/drive/MyDrive/Capstone Project/imdb_data"

# Modify the query to use local file paths instead of URLs
query = f"""
WITH title_data AS (
    SELECT -- these are all the columns I want to include from
        tb.tconst,
        tb.originalTitle,
        tb.runtimeMinutes,
        y.genre, --All Columns from title.basics
        tp.category,
        tr.averageRating,
        tn.region
    FROM read_csv_auto('{download_dir}/title.basics.tsv.gz', compression='gzip', sep='\t', nullstr='\\N') tb

    -- Explode comma-separated genres into individual rows
    LEFT JOIN LATERAL (
        SELECT UNNEST(STRING_SPLIT(tb.genres, ',')) AS genre
    ) AS y ON TRUE

    LEFT JOIN read_csv_auto('{download_dir}/title.crew.tsv.gz', compression='gzip', sep='\t', nullstr='\\N') tc
         ON tb.tconst = tc.tconst
    LEFT JOIN read_csv_auto('{download_dir}/title.principals.tsv.gz', compression='gzip', sep='\t', nullstr='\\N') tp
        ON tb.tconst = tp.tconst
    LEFT JOIN read_csv_auto('{download_dir}/title.ratings.tsv.gz', compression='gzip', sep='\t', nullstr='\\N') tr
        ON tb.tconst = tr.tconst
    LEFT JOIN read_csv_auto('{download_dir}/title.akas.tsv.gz', compression='gzip', sep='\t', nullstr='\\N') tn
        ON tb.tconst = tn.titleID

    -- Only include movies
    WHERE tb.titleType = 'movie'
 ),

name_data AS (
    SELECT
        nb.primaryName,
        nb.PrimaryProfession,
        g.tconst2,
        h.Primary
    FROM read_csv_auto('{download_dir}/name.basics.tsv.gz', compression='gzip', sep='\t', nullstr='\\N') nb,

    -- Explode knownForTitles and primaryProfession columns
    LATERAL (SELECT UNNEST(STRING_SPLIT(nb.knownForTitles, ',')) AS tconst2) AS g,
    LATERAL (SELECT UNNEST(STRING_SPLIT(nb.primaryProfession, ',')) AS Primary) AS h
)

-- Final SELECT and join
SELECT
    tb.*,
    nb.primaryName,
    nb.Primary as primaryProfessionSplit,
    nb.tconst2 as knownForTitleSplit
FROM title_data tb
LEFT JOIN name_data nb
    ON tb.tconst = nb.tconst2
LIMIT 3000000
"""

# Step 3: Run the query with LIMIT inside
# Run the DuckDB SQL query
con.execute(query)

# Step 4: Fetch result into pandas
# Fetch result into a pandas DataFrame for further analysis
df = con.fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
print(df.shape)

(3000000, 10)


In [ ]:
print(df.columns.tolist())

['tconst', 'originalTitle', 'runtimeMinutes', 'genre', 'category', 'averageRating', 'region', 'primaryName', 'primaryProfessionSplit', 'knownForTitleSplit']


In [ ]:
print(df.head())

      tconst originalTitle  runtimeMinutes    genre  category  averageRating  \
0  tt0000009    Miss Jerry              45  Romance   actress            5.4   
1  tt0000009    Miss Jerry              45  Romance     actor            5.4   
2  tt0000009    Miss Jerry              45  Romance     actor            5.4   
3  tt0000009    Miss Jerry              45  Romance  director            5.4   
4  tt0000009    Miss Jerry              45  Romance    writer            5.4   

  region        primaryName primaryProfessionSplit knownForTitleSplit  
0     DE  William Courtenay                  actor          tt0000009  
1     DE  William Courtenay                  actor          tt0000009  
2     DE  William Courtenay                  actor          tt0000009  
3     DE  William Courtenay                  actor          tt0000009  
4     DE  William Courtenay                  actor          tt0000009  


In [ ]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   tconst                  object 
 1   originalTitle           object 
 2   runtimeMinutes          Int64  
 3   genre                   object 
 4   category                object 
 5   averageRating           float64
 6   region                  object 
 7   primaryName             object 
 8   primaryProfessionSplit  object 
 9   knownForTitleSplit      object 
dtypes: Int64(1), float64(1), object(8)
memory usage: 231.7+ MB
None


This line saves your full DataFrame as a Parquet file, which is:
Columnar and highly efficient for analytics.
Great for loading back into Pandas, Spark, or DuckDB quickly.
Stored on Google Drive under your Capstone Project folder.

if you're working with large datasets, Parquet is preferred over CSV due to better compression and faster read/write times.

In [ ]:
df.to_parquet("/content/drive/MyDrive/Capstone Project/imdb_ml_data3M.parquet", index=False) #file output location
print("✅ Data saved as Parquet to Google Drive.")


✅ Data saved as Parquet to Google Drive.


In [ ]:
import pandas as pd
df = pd.read_parquet("//content/drive/MyDrive/Capstone Project/imdb_ml_data3M.parquet")
df.head(100)

,tconst,originalTitle,runtimeMinutes,genre,category,averageRating,region,primaryName,primaryProfessionSplit,knownForTitleSplit
0,tt0000009,Miss Jerry,45,Romance,actress,5.4,DE,William Courtenay,actor,tt0000009
1,tt0000009,Miss Jerry,45,Romance,actor,5.4,DE,William Courtenay,actor,tt0000009
2,tt0000009,Miss Jerry,45,Romance,actor,5.4,DE,William Courtenay,actor,tt0000009
3,tt0000009,Miss Jerry,45,Romance,director,5.4,DE,William Courtenay,actor,tt0000009
4,tt0000009,Miss Jerry,45,Romance,writer,5.4,DE,William Courtenay,actor,tt0000009
...,...,...,...,...,...,...,...,...,...,...
95,tt0000574,The Story of the Kelly Gang,70,Adventure,actor,6.0,RS,Sam Crewes,art_department,tt0000574
96,tt0000574,The Story of the Kelly Gang,70,Adventure,actress,6.0,RS,Sam Crewes,art_department,tt0000574
97,tt0000574,The Story of the Kelly Gang,70,Adventure,actor,6.0,RS,Sam Crewes,art_department,tt0000574
98,tt0000574,The Story of the Kelly Gang,70,Adventure,actor,6.0,RS,Sam Crewes,art_department,tt0000574


In [ ]:
filtered_df2 = df[df['originalTitle'] == 'The Ring']
filtered_df2.drop(columns=['averageRating']) #removing the rating column before using it to rest the model.


,tconst,originalTitle,runtimeMinutes,genre,category,region,primaryName,primaryProfessionSplit,knownForTitleSplit
1162956,tt0018328,The Ring,89,Sport,actor,FR,Carl Brisson,archive_footage,tt0018328
1162957,tt0018328,The Ring,89,Sport,actress,FR,Carl Brisson,archive_footage,tt0018328
1162958,tt0018328,The Ring,89,Sport,actor,FR,Carl Brisson,archive_footage,tt0018328
1162959,tt0018328,The Ring,89,Sport,actor,FR,Carl Brisson,archive_footage,tt0018328
1162960,tt0018328,The Ring,89,Sport,actor,FR,Carl Brisson,archive_footage,tt0018328
...,...,...,...,...,...,...,...,...,...
1516335,tt0018328,The Ring,89,Drama,actress,None,Lillian Hall-Davis,actress,tt0018328
1516336,tt0018328,The Ring,89,Drama,actor,None,Lillian Hall-Davis,actress,tt0018328
1516337,tt0018328,The Ring,89,Drama,director,None,Lillian Hall-Davis,actress,tt0018328
1516338,tt0018328,The Ring,89,Drama,writer,None,Lillian Hall-Davis,actress,tt0018328


Random Forest

In [ ]:
df_sample = df.sample(n=3000000, random_state=42)
#This randomly samples 3 million rows from your dataset to reduce memory usage and speed up training.
  #random_state=42 ensures reproducibility.

# Step 3: Drop rows where target is missing
  #Ensures that all rows have a value for the averageRating column — your prediction target.
df_ml = df_sample[df_sample['averageRating'].notna()]

# Step 4: Define target and features
  #You select relevant feature columns and your target (what you want to predict).
  #Rows with missing feature values are dropped.
target = 'averageRating'
features = [
    'runtimeMinutes',
    'tconst',
    'genre',
    'originalTitle',
    'category',
    'primaryName'
]
df_ml = df_ml[features + [target]].dropna()

# Step 5: Train/test split
  #Splits the data into: 80% training data, 20% testing data, random_state=42 ensures consistent splits.
X = df_ml[features]
y = df_ml[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Preprocessing and model pipeline
  #Numerical feature: fill missing runtimeMinutes with the mean.
  #Categorical features: use OneHotEncoder, ignore unseen categories during inference.
categorical = ['tconst','genre', 'originalTitle', 'category', 'primaryName']
numerical = ['runtimeMinutes']

preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='mean'), numerical),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
])

#Combines preprocessing and model training into a single Pipeline.
#Uses a Random Forest Regressor with 100 trees.
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Step 7: Train the model
  #Generates predictions on the test set.
  #Evaluates performance using:
  #RMSE (Root Mean Squared Error): lower is better.
  #R² score: closer to 1.0 is better (explained variance).
model.fit(X_train, y_train)

# Step 8: Predict and evaluate
y_pred = model.predict(X_test)
print(y_pred)
print("RMSE:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
#Step1: Extract Feature Importances
  #Accesses the feature_importances_ attribute from the RandomForestRegressor.
  #This gives a numeric score representing how much each feature contributes to prediction performance.
importances = model.named_steps['regressor'].feature_importances_

#Step 2: Get Transformed Features
  #Fetches all feature names after preprocessing, including all one-hot encoded categories.
  #Ensures the importances line up with the actual input features used by the model.
feature_names = model.named_steps['preprocessor'].get_feature_names_out()

#Step 3: Create a DF to analyze importance
  #Combines feature names and their importance scores into a Pandas DataFrame for easy inspection.
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})

#Step 4: View Top 10 Most Important Features
  #Sorts features by importance in descending order.
  #Displays the top 10 features contributing most to your model’s predictions.
print(importance_df.sort_values(by='importance', ascending=False).head(10))